In [2]:
import os
import pandas as pd


In [22]:
import os
import pandas as pd

# Caminho base para o diretório experiments
caminho_base = "./experiments"

# Inicializa contadores globais
total_celulas = 0
total_faltosos = 0
faltosos_continuos = 0
faltosos_semi_estruturados = 0

# Navega pela estrutura de diretórios
for pasta_principal in ['10', '20', '30']:
    caminho_principal = os.path.join(caminho_base, pasta_principal)
    for subpasta in ['48', '512', '1024']:
        caminho_subpasta = os.path.join(caminho_principal, subpasta)
        for tipo in ['teste', 'treino']:
            caminho_tipo = os.path.join(caminho_subpasta, tipo)
            caminho_input = os.path.join(caminho_tipo, 'input')  # Apenas a subpasta 'input'
            
            if os.path.exists(caminho_input):  # Verifica se a pasta 'input' existe
                for arquivo in os.listdir(caminho_input):
                    if arquivo.endswith('.csv'):
                        caminho_arquivo = os.path.join(caminho_input, arquivo)
                        # print(f"Processando: {caminho_arquivo}")
                        
                        # Lê o CSV
                        df = pd.read_csv(caminho_arquivo)
                        
                        # Atualiza o total de células
                        total_celulas += df.size  # Número de células (linhas × colunas)

                        # Verifica dados faltosos por coluna (série)
                        for coluna in df.columns:
                            serie = df[coluna].isna()  # Detecta valores faltosos na coluna

                            # Dados faltosos contínuos
                            continuos = (serie != serie.shift()).cumsum()  # Grupos consecutivos
                            faltosos_continuos += serie.groupby(continuos).sum().gt(1).sum()

                            # Semi-Estruturados (distância entre NaNs <= 1)
                            nan_indices = serie[serie].index  # Índices dos NaNs
                            if len(nan_indices) > 1:  # Verifica pares consecutivos de NaNs
                                distancias = nan_indices[1:] - nan_indices[:-1]
                                faltosos_semi_estruturados += (distancias <= 2).sum()

                            # Atualiza o total de NaNs
                            total_faltosos += serie.sum()

# Calcula porcentagens globais
if total_faltosos > 0:
    porcentagem_continuos = (faltosos_continuos / total_faltosos) * 100
    porcentagem_semi_estruturados = (faltosos_semi_estruturados / total_faltosos) * 100
    porcentagem_nao_continuos = 100 - (porcentagem_continuos + porcentagem_semi_estruturados)
else:
    porcentagem_continuos = porcentagem_semi_estruturados = porcentagem_nao_continuos = 0

# Exibe os resultados
print(f"Total Células: {total_celulas:.2f}")
print(f"Porcentagem Dados Faltosos Contínuos: {porcentagem_continuos:.2f}%")
print(f"Porcentagem Dados Faltosos Semi-Estruturados: {porcentagem_semi_estruturados:.2f}%")
print(f"Porcentagem Dados Faltosos Não Contínuos: {porcentagem_nao_continuos:.2f}%")


Total Células: 507459420.00
Porcentagem Dados Faltosos Contínuos: 17.00%
Porcentagem Dados Faltosos Semi-Estruturados: 39.56%
Porcentagem Dados Faltosos Não Contínuos: 43.44%
